In [23]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [2]:
nba = pd.read_csv('nba.csv', low_memory=False)

## Original data

In [3]:
pd.set_option('display.max_rows', 90)
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5


## First normal form

In [4]:
nba = nba.drop(['Inactives'], axis=1)
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5


## Second normal form

In [5]:
players = nba[['player_id', 'player']].drop_duplicates()
players

,player_id,player
0,kispeco01,Corey Kispert
1,kuzmaky01,Kyle Kuzma
2,caldwke01,Kentavious Caldwell-Pope
3,netora01,Raul Neto
4,bryanth01,Thomas Bryant
...,...,...
109702,frazimi01,Michael Frazier
110441,howarwi01,William Howard
110913,mbahalu01,Luc Mbah a Moute
111399,bowmaky01,Ky Bowman


In [6]:
nba = nba.drop(['player'], axis=1)

In [7]:
games = nba[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()
games

,game_id,game_date,OT,season
0,202202170BRK,2022-02-17,0,2022
26,202202170CHO,2022-02-17,2,2022
48,202202170LAC,2022-02-17,0,2022
71,202202170MIL,2022-02-17,0,2022
95,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
108259,202001080GSW,2020-01-08,0,2020
108887,202008020HOU,2020-08-02,0,2020
109683,201911060HOU,2019-11-06,0,2020
110125,201912250GSW,2019-12-25,0,2020


In [8]:
nba = nba.drop(['game_date', 'OT', 'season'], axis=1)

In [9]:
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5
Team_orb_pct,22.9,22.9,22.9
Team_ft_rate,0.157,0.157,0.157


## Third normal form

In [10]:
nba.columns

Index(['Unnamed: 0', 'game_id', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'player_id',
       'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft',
       'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct',
       'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg',
       'def_rtg', 'bpm', 'minutes', 'double_double', 'triple_double', 'DKP',
       'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting',
       'last_60_minutes_per_game_be

In [11]:
team_game = nba[['game_id', 'Team_Abbrev', 'H_A', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

In [12]:
team_game.head(4).T

,0,13,26,37
game_id,202202170BRK,202202170BRK,202202170CHO,202202170CHO
Team_Abbrev,WAS,BRK,MIA,CHO
H_A,A,H,A,H
Team_Score,117,103,111,107
Team_pace,94.5,94.5,88.8,88.8
Team_efg_pct,0.627,0.483,0.471,0.453
Team_tov_pct,13.5,13.1,11.1,13.6
Team_orb_pct,22.9,33.3,26.8,28.1
Team_ft_rate,0.157,0.191,0.147,0.221
Team_off_rtg,123.8,109.0,103.4,99.7


In [13]:
player_game = nba.drop(['Unnamed: 0', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg'],
                      axis=1)

In [14]:
team_game

,game_id,Team_Abbrev,H_A,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202202170BRK,WAS,A,117,94.5,0.627,13.5,22.9,0.157,123.8,BRK
13,202202170BRK,BRK,H,103,94.5,0.483,13.1,33.3,0.191,109.0,WAS
26,202202170CHO,MIA,A,111,88.8,0.471,11.1,26.8,0.147,103.4,CHO
37,202202170CHO,CHO,H,107,88.8,0.453,13.6,28.1,0.221,99.7,MIA
48,202202170LAC,HOU,A,111,103.7,0.533,15.3,24.0,0.154,107.1,LAC
...,...,...,...,...,...,...,...,...,...,...,...
112068,202002270GSW,GSW,H,86,104.8,0.481,23.6,12.2,0.113,82.1,LAL
112079,202002290PHO,GSW,A,115,98.6,0.523,9.0,28.9,0.276,116.6,PHO
112090,202003010GSW,GSW,H,110,100.2,0.522,17.4,38.3,0.191,109.8,WAS
112101,202003030DEN,GSW,A,116,94.4,0.622,10.7,12.8,0.171,122.9,DEN


In [15]:
player_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112123 entries, 0 to 112122
Data columns (total 59 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   game_id                            112123 non-null  object 
 1   player_id                          112123 non-null  object 
 2   starter                            112123 non-null  int64  
 3   mp                                 112123 non-null  object 
 4   fg                                 112123 non-null  int64  
 5   fga                                112123 non-null  int64  
 6   fg_pct                             112123 non-null  float64
 7   fg3                                112123 non-null  int64  
 8   fg3a                               112123 non-null  int64  
 9   fg3_pct                            112123 non-null  float64
 10  ft                                 112123 non-null  int64  
 11  fta                                1121

In [16]:
team_game.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6394 entries, 0 to 112112
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game_id          6394 non-null   object 
 1   Team_Abbrev      6394 non-null   object 
 2   H_A              6394 non-null   object 
 3   Team_Score       6394 non-null   int64  
 4   Team_pace        6394 non-null   float64
 5   Team_efg_pct     6394 non-null   float64
 6   Team_tov_pct     6394 non-null   float64
 7   Team_orb_pct     6394 non-null   float64
 8   Team_ft_rate     6394 non-null   float64
 9   Team_off_rtg     6394 non-null   float64
 10  Opponent_Abbrev  6394 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 599.4+ KB


,game_id,Team_Abbrev,H_A,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202202170BRK,WAS,A,117,94.5,0.627,13.5,22.9,0.157,123.8,BRK
13,202202170BRK,BRK,H,103,94.5,0.483,13.1,33.3,0.191,109.0,WAS
26,202202170CHO,MIA,A,111,88.8,0.471,11.1,26.8,0.147,103.4,CHO
37,202202170CHO,CHO,H,107,88.8,0.453,13.6,28.1,0.221,99.7,MIA
48,202202170LAC,HOU,A,111,103.7,0.533,15.3,24.0,0.154,107.1,LAC
...,...,...,...,...,...,...,...,...,...,...,...
112068,202002270GSW,GSW,H,86,104.8,0.481,23.6,12.2,0.113,82.1,LAL
112079,202002290PHO,GSW,A,115,98.6,0.523,9.0,28.9,0.276,116.6,PHO
112090,202003010GSW,GSW,H,110,100.2,0.522,17.4,38.3,0.191,109.8,WAS
112101,202003030DEN,GSW,A,116,94.4,0.622,10.7,12.8,0.171,122.9,DEN


In [30]:
dbserver = psycopg2.connect(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = '5432'
)
dbserver.autocommit = True

In [31]:
cursor = dbserver.cursor()

In [32]:
try:
    cursor.execute('CREATE DATABASE nba')
except:
    cursor.execute('DROP DATABASE nba')
    cursor.execute('CREATE DATABASE nba')

In [33]:
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}/{db}".format(
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    db = 'nba'
))

In [53]:
games.columns = [x.lower() for x in games.columns]
players.columns = [x.lower() for x in players.columns]
team_game.columns = [x.lower() for x in team_game.columns]
player_game.columns = [x.lower() for x in player_game.columns]

In [54]:
games.to_sql('games', con=engine, index=False, chunksize=1000, if_exists='replace')

3197

In [55]:
players.to_sql('players', con=engine, index=False, chunksize=1000, if_exists='replace')

812

In [56]:
team_game.to_sql('team_game', con=engine, index=False, chunksize=1000, if_exists='replace')

6394

In [59]:
player_game.to_sql('player_game', con=engine, index=False, chunksize=500, if_exists='replace')

112123

In [58]:
myquery = '''
SELECT *
FROM games
'''
pd.read_sql_query(myquery, con=engine)

,game_id,game_date,ot,season
0,202202170BRK,2022-02-17,0,2022
1,202202170CHO,2022-02-17,2,2022
2,202202170LAC,2022-02-17,0,2022
3,202202170MIL,2022-02-17,0,2022
4,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
3192,202001080GSW,2020-01-08,0,2020
3193,202008020HOU,2020-08-02,0,2020
3194,201911060HOU,2019-11-06,0,2020
3195,201912250GSW,2019-12-25,0,2020


In [44]:
myquery = '''
SELECT *
FROM games
WHERE game_date = '2021-04-12'
'''
pd.read_sql_query(myquery, con=engine)

,game_id,game_date,OT,season
0,202104120GSW,2021-04-12,0,2021
1,202104120NYK,2021-04-12,0,2021
2,202104120ORL,2021-04-12,0,2021
3,202104120UTA,2021-04-12,0,2021
4,202104120DAL,2021-04-12,0,2021
5,202104120NOP,2021-04-12,0,2021
6,202104120MEM,2021-04-12,0,2021
7,202104120PHO,2021-04-12,0,2021


In [47]:
player_game

,game_id,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,kispeco01,1,32:30,6,9,0.667,4,6,0.667,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,kuzmaky01,1,30:16,2,7,0.286,0,3,0.000,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,caldwke01,1,25:26,3,7,0.429,1,3,0.333,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,netora01,1,20:40,5,7,0.714,1,1,1.000,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,bryanth01,1,14:04,5,6,0.833,0,1,0.000,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112118,202003070GSW,wiggian01,1,37:04:00,3,10,0.300,0,0,0.000,...,0.107914,13.08,33.110667,19.232562,0.0,2.0,77.0,21.0,0.0,57.207786
112119,202003070GSW,toscaju01,1,27:43:00,3,6,0.500,0,2,0.000,...,0.036079,6.00,25.470833,20.228571,5.0,45.0,43.0,7.0,0.0,58.202391
112120,202003070GSW,bendedr01,0,13:15,4,4,1.000,2,2,1.000,...,0.150943,4.00,24.083333,13.228788,0.0,0.0,0.0,9.0,91.0,49.630640
112121,202003070GSW,muldemy01,1,31:48:00,5,10,0.500,3,7,0.429,...,0.094340,12.64,34.783333,27.691667,0.0,44.0,48.0,8.0,0.0,58.923515


In [48]:
# all games where a player scores > 40, and team > 120, and calculate % of team scores

In [63]:
myquery = '''
SELECT pg.game_id, pg.player_id, pg.pts, tg.team_score,
    point_percent = pg.pts/tg.team_score 
FROM player_game pg
INNER JOIN team_game tg
    ON pg.game_id = tg.game_id
WHERE pg.pts > 40 AND tg.team_score > 120
'''
pd.read_sql_query(myquery, con=engine)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "point_percent" does not exist
LINE 3:     point_percent = pg.pts/tg.team_score 
            ^

[SQL: 
SELECT pg.game_id, pg.player_id, pg.pts, tg.team_score,
    point_percent = pg.pts/tg.team_score 
FROM player_game pg
INNER JOIN team_game tg
    ON pg.game_id = tg.game_id
WHERE pg.pts > 40 AND tg.team_score > 120
]
(Background on this error at: https://sqlalche.me/e/20/f405)